In [1]:
import numpy as np
import pandas as pd
import scipy.optimize as optimize
from fffit.utils import (
    shuffle_and_split,
    values_scaled_to_real,
)
import sys
sys.path.append("../")


from utils.r143a import R143aConstants
R143a = R143aConstants()

In [2]:
top_liq = pd.read_csv("../csv/r143a-density-iter1-liquid-params.csv", delimiter = ",", index_col = 0)
top_vap = pd.read_csv("../csv/r143a-density-iter1-vapor-params.csv", delimiter = ",", index_col = 0)

In [3]:
top_liq = top_liq.reset_index(drop=True)
top_vap = top_vap.reset_index(drop=True)

dist_guess = 1
dist_seed = 10
bounds = [(0,None)]
target_num = 100

In [4]:
def opt_dist(distance, top_samples, constants, target_num, rand_seed = None, eval = False):
    """
    Calculates the distance between points such that exactly a target number of points are chosen for the next iteration
    
    Parameters:
    -----------
        distance: float, The allowable minimum distance between points
        top_samples: pandas data frame, Collection of top liquid/vapor sampes
        constants: utils.r143a.R143aConstants, contains the infromation for a certain refrigerant
        target_num: int, the number of samples to choose next
        rand_seed: int, the seed number to use: None by default
        eval: bool, Determines whether error is calculated or new_points is returned
    
    Returns:
        error: float, The squared error between the target value and number of new_points
        OR
        new_points: pandas data frame, a pandas data frame containing the number of points to be used 
    """
    if rand_seed != None:
        np.random.seed(rand_seed)
    new_points = pd.DataFrame()
    discarded_points = pd.DataFrame(columns=top_samples.columns)
    while len(top_samples > 0):
        # Shuffle the pareto points
        top_samples = top_samples.sample(frac=1)
        new_points = new_points.append(top_samples.iloc[[0]])
        # Remove anything within distance
        l1_norm = np.sum(
            np.abs(
                top_samples[list(constants.param_names)].values
                - new_points[list(constants.param_names)].iloc[[-1]].values
            ),
            axis=1,
        )
        points_to_remove = np.where(l1_norm < distance)[0]
        discarded_points = discarded_points.append(
            top_samples.iloc[points_to_remove]
        )
        top_samples.drop(
            index=top_samples.index[points_to_remove], inplace=True
        )
        error = (target_num - len(new_points))**2
    if eval == True:
        return new_points
    else:
        return error

In [5]:
args_v = (top_vap ,R143a, target_num, dist_seed)
solution_v = optimize.minimize(opt_dist, dist_guess, bounds = bounds, args=args_v, method='Nelder-Mead')
dist_opt_v = solution_v.x
new_points_v = opt_dist(dist_opt_v, top_vap, R143a, target_num, rand_seed=dist_seed , eval = True)

while len(new_points_v) != target_num:
    dist_opt_v = solution_v.x
    dist_seed += 1
    new_points_v = opt_dist(dist_opt_v, top_vap, R143a, target_num, rand_seed=dist_seed , eval = True)
    
print(len(new_points_v), "top vapor density points are left after removing similar points using a distance of"
      , np.round(dist_opt_v,5))
new_points_v

100 top vapor density points are left after removing similar points using a distance of [0.83125]


,sigma_C1,sigma_C2,sigma_F1,sigma_H1,epsilon_C1,epsilon_C2,epsilon_F1,epsilon_H1
16,0.975353,0.290719,0.580944,0.197721,0.312727,0.347546,0.456071,0.992134
122,0.113193,0.515903,0.357673,0.848227,0.703194,0.486292,0.526738,0.945810
146,0.150932,0.058594,0.457216,0.663563,0.850335,0.861994,0.428529,0.899232
77,0.607174,0.380483,0.449415,0.835516,0.553293,0.245617,0.645935,0.977117
254,0.182102,0.223335,0.394433,0.959247,0.954313,0.899046,0.805838,0.243042
...,...,...,...,...,...,...,...,...
161,0.949339,0.132454,0.643023,0.100938,0.110031,0.676661,0.509105,0.867131
155,0.970235,0.476779,0.393574,0.581229,0.684531,0.072088,0.145180,0.978712
187,0.732970,0.817605,0.444647,0.325993,0.840264,0.008624,0.153378,0.922369
53,0.811071,0.814599,0.227812,0.887362,0.921455,0.035080,0.010206,0.868692


In [6]:
args_l = (top_liq ,R143a, target_num, dist_seed)
solution_l = optimize.minimize(opt_dist, dist_guess, bounds = bounds, args=args_l, method='Nelder-Mead')
dist_opt_l = solution_l.x
new_points_l = opt_dist(dist_opt_l, top_liq, R143a, target_num, rand_seed=dist_seed , eval = True)

while len(new_points_v) != target_num:
    dist_opt_l = solution_l.x
    dist_seed += 1
    new_points_l = opt_dist(dist_opt_l, top_vap, R143a, target_num, rand_seed=dist_seed , eval = True)
    
print(len(new_points_l), "top liquid density points are left after removing similar points using a distance of"
      , np.round(dist_opt_l,5))
new_points_l

100 top liquid density points are left after removing similar points using a distance of [1.35313]


,sigma_C1,sigma_C2,sigma_F1,sigma_H1,epsilon_C1,epsilon_C2,epsilon_F1,epsilon_H1
5333,0.992720,0.974989,0.554406,0.177801,0.933232,0.256004,0.038790,0.435341
1730,0.475796,0.754183,0.488438,0.742289,0.326563,0.599589,0.861474,0.377545
1815,0.554732,0.272612,0.656282,0.273564,0.863030,0.920344,0.092892,0.907676
549,0.970441,0.978472,0.482197,0.398935,0.259347,0.850313,0.120198,0.508305
3555,0.801411,0.007867,0.596508,0.751199,0.911319,0.488179,0.470999,0.713246
...,...,...,...,...,...,...,...,...
4584,0.327955,0.043514,0.808225,0.022027,0.962676,0.966998,0.728053,0.880795
4202,0.966463,0.486723,0.692082,0.173158,0.984482,0.486470,0.003934,0.856087
4020,0.200015,0.590715,0.399516,0.865142,0.739294,0.954016,0.052524,0.952886
4111,0.271689,0.972848,0.320754,0.908850,0.685028,0.259682,0.926029,0.254757


In [7]:
# Save to CSV
# new_points.to_csv(csv_path + out_csv_name)
csv_path = "Test/"
iternum=1
out_top_liquid_csv_name = "r143a-density-iter" + str(iternum ) + "-liquid-params.csv"
out_top_vapor_csv_name = "r143a-density-iter" + str(iternum ) + "-vapor-params.csv"
print(type(new_points_l))
# new_points_l = pd.concat(new_points_l)
new_points_l.to_csv(csv_path + out_top_liquid_csv_name)
# new_points_v = pd.concat(new_points_v)
new_points_v.to_csv(csv_path + out_top_vapor_csv_name)
print(new_points_l)

<class 'pandas.core.frame.DataFrame'>
      sigma_C1  sigma_C2  sigma_F1  sigma_H1  epsilon_C1  epsilon_C2  \
5333  0.992720  0.974989  0.554406  0.177801    0.933232    0.256004   
1730  0.475796  0.754183  0.488438  0.742289    0.326563    0.599589   
1815  0.554732  0.272612  0.656282  0.273564    0.863030    0.920344   
549   0.970441  0.978472  0.482197  0.398935    0.259347    0.850313   
3555  0.801411  0.007867  0.596508  0.751199    0.911319    0.488179   
...        ...       ...       ...       ...         ...         ...   
4584  0.327955  0.043514  0.808225  0.022027    0.962676    0.966998   
4202  0.966463  0.486723  0.692082  0.173158    0.984482    0.486470   
4020  0.200015  0.590715  0.399516  0.865142    0.739294    0.954016   
4111  0.271689  0.972848  0.320754  0.908850    0.685028    0.259682   
1873  0.841919  0.821235  0.099773  0.994933    0.080162    0.974995   

      epsilon_F1  epsilon_H1  
5333    0.038790    0.435341  
1730    0.861474    0.377545  
1815